In [ ]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-02'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
df=pd.read_sql_query('Select Distinct * from allmatches',con=devengine)
# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]

# Populate Set Winner columns based on set scores and Completed_Sets
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    w_col = f"w{set_num}"
    l_col = f"l{set_num}"
    
    # Ensure the column exists
    if w_col in df.columns and l_col in df.columns:
        # Determine the winner for the set
        df[set_winner_col] = df.apply(
            lambda row: row["Player_1"] if row[w_col] > row[l_col] else row["Player_2"], axis=1
        )
    else:
        df[set_winner_col] = None  # Initialize with None if scores don't exist

    # Clear values for sets beyond the number of completed sets
    df[set_winner_col] = df.apply(
        lambda row: row[set_winner_col] if set_num <= row["Completed_Sets"] else None, axis=1
    )
df["fav_odds"] = df.apply(
    lambda row: min(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
df["dog_odds"] = df.apply(
    lambda row: max(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
# Create columns for Fav and Dog based on odds
df["Fav"] = df.apply(
    lambda row: row["Player_1"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_2"], axis=1
)
df["Dog"] = df.apply(
    lambda row: row["Player_2"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_1"], axis=1
)

# Adjust the Set Winner columns to reflect the favorite and underdog
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    
    # Reorganize the Set Winner columns based on the favorite and underdog
    df[set_winner_col] = df.apply(
        lambda row: row["Fav"] if row[set_winner_col] == row["Player_1"] else row["Dog"], axis=1
    )

# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]


df["Match_Type"] = df.apply(
    lambda row: "Best of 5" if row["Winner_Sets"] + row["Loser_Sets"] > 3 else "Best of 3", axis=1
)
# Loop through Set 1 Winner to Set 5 Winner columns
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    set_winner_fav_col = f"Set {set_num} Winner_Fav"
    
    # Check if the necessary columns are present in the DataFrame
    if "Fav" in df.columns and "Dog" in df.columns:
        # Create new columns Set X Winner_Fav based on the logic
        df[set_winner_fav_col] = df.apply(
            lambda row: 'Fav' if row[set_winner_col] == row["Fav"] else 'Dog', axis=1
        )
    else:
        df[set_winner_fav_col] = None  # Initialize with None if "Fav" or "Dog" columns don't exist

#df=df[df['Set 2 Winner'].notna()]
#df=df[df['Set 1 Winner_Fav']=='Dog']
#df=df[df['Set 3 Winner_Fav']=='Dog']
#df[['Date','Set 1 Winner_Fav','Set 1 Winner','w1','l1','Set 2 Winner','Set 2 Winner_Fav','w2','l2','Set 3 Winner','Set 3 Winner_Fav','w3','l3','Fav','Dog','fav_odds']][df['w3']!=" "]
df['Winner']=df['Player_1']

In [ ]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-02'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
df=pd.read_sql_query('Select Distinct * from allmatches where surface like "clay"',con=devengine)
# Ensure Winner_Sets and Loser_Sets are integers

# Example DataFrame: df
set_cols = [('w1', 'l1'), ('w2', 'l2'), ('w3', 'l3'), ('w4', 'l4'), ('w5', 'l5')]

# Add columns to record set winners
for i, (w_col, l_col) in enumerate(set_cols, start=1):
    def determine_set_winner(row):
        w_games = row[w_col]
        l_games = row[l_col]
        if pd.isna(w_games) or pd.isna(l_games):
            return np.nan  # Set not played
        if w_games > l_games:
            return row['Player_1']
        elif w_games < l_games:
            return row['Player_2']
        else:
            return 'Tie'  # Rare, but possible in corrupted data

    df[f'set_{i}_winner'] = df.apply(determine_set_winner, axis=1)

df['Winner']=df['Player_1']
clean=df[['Player_1','Player_2','Player_1_Odds','Player_2_Odds','set_1_winner','set_2_winner','set_3_winner','Surface','Date','Winner','Sex']]
clean=clean[clean['set_1_winner']!='Tie']
data=clean[clean['set_2_winner']!='Tie'].copy()


data['Set12']=data['set_1_winner']==data['set_2_winner']

def get_fav_dog(row):
    if row['Player_1_Odds'] < row['Player_2_Odds']:
        return pd.Series([row['Player_1_Odds'], row['Player_2_Odds'], row['Player_1'], row['Player_2']],
                         index=['fav_odds', 'dog_odds', 'fav', 'dog'])
    else:
        return pd.Series([row['Player_2_Odds'], row['Player_1_Odds'], row['Player_2'], row['Player_1']],
                         index=['fav_odds', 'dog_odds', 'fav', 'dog'])

# Apply to your dataframe
data[['fav_odds', 'dog_odds', 'Fav', 'Dog']] = data.apply(get_fav_dog, axis=1)
data['Fav_Win'] = data['Winner']==data['Fav']
data['Dog_Win'] = data['Winner']!=data['Fav']
data['Fav_Odds']=data[['Player_1_Odds','Player_2_Odds']].min(axis=1)
data['Dog_Odds']=data[['Player_1_Odds','Player_2_Odds']].max(axis=1)


In [ ]:
calc=data.copy()
import pandas as pd

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the last 30 days, excluding the current match
def wins_last_30_before_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened before the cutoff date within the last 30 days
        past_wins = group[(group['Date'] < cutoff) & (group['Date'] >= cutoff - pd.Timedelta(days=30))]
        result.append(len(past_wins))  # Count the wins in the past 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the last 30 days for each player before the match
player_wins['Wins_Last_30_Days_Before'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_before_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "before match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (before the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Fav_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Merge back for Dog (before the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Dog_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no prior wins)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


In [51]:
stake=100
date_filter=calc[calc['Date']>'2023-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']<1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter=oddsfilter[oddsfilter['Fav']==oddsfilter['set_1_winner']]
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
oddsfilter#.to_csv('_ClayResults_Womens.csv')

Total Profit: -15472.0 259 0.10810810810810811


,Player_1,Player_2,Player_1_Odds,Player_2_Odds,set_1_winner,set_2_winner,set_3_winner,Surface,Date,Winner,...,dog_odds,Fav,Dog,Fav_Win,Dog_Win,Fav_Odds,Dog_Odds,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before,Profit
201,Fernanda Contreras Gomez,Marine Partaud,1.31,3.26,Fernanda Contreras Gomez,Marine Partaud,Fernanda Contreras Gomez,Clay,2023-01-31,Fernanda Contreras Gomez,...,3.26,Fernanda Contreras Gomez,Marine Partaud,True,False,1.31,3.26,0,1,-100.0
225,Emiliana Arango,Martina Colmegna,1.29,3.46,Emiliana Arango,Emiliana Arango,Tie,Clay,2023-02-04,Emiliana Arango,...,3.46,Emiliana Arango,Martina Colmegna,True,False,1.29,3.46,2,3,-100.0
702,Hailey Baptiste,Quinn Gleason,1.32,3.19,Hailey Baptiste,Hailey Baptiste,Tie,Clay,2023-03-28,Hailey Baptiste,...,3.19,Hailey Baptiste,Quinn Gleason,True,False,1.32,3.19,0,1,-100.0
740,Nadia Podoroska,Paula Ormaechea,1.37,3.01,Nadia Podoroska,Nadia Podoroska,Tie,Clay,2023-03-29,Nadia Podoroska,...,3.01,Nadia Podoroska,Paula Ormaechea,True,False,1.37,3.01,0,3,-100.0
838,Fernanda Contreras Gomez,Martina Colmegna,1.21,4.13,Fernanda Contreras Gomez,Fernanda Contreras Gomez,Tie,Clay,2023-04-02,Fernanda Contreras Gomez,...,4.13,Fernanda Contreras Gomez,Martina Colmegna,True,False,1.21,4.13,0,3,-100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18825,Oleksandra Oliynykova,Eva Vedder,1.32,3.23,Oleksandra Oliynykova,Eva Vedder,Oleksandra Oliynykova,Clay,2025-06-09,Oleksandra Oliynykova,...,3.23,Oleksandra Oliynykova,Eva Vedder,True,False,1.32,3.23,0,1,-100.0
19481,Ekaterina Kazionova,Caroline Werner,4.34,1.19,Caroline Werner,Ekaterina Kazionova,Ekaterina Kazionova,Clay,2025-07-07,Ekaterina Kazionova,...,4.34,Caroline Werner,Ekaterina Kazionova,False,True,1.19,4.34,0,3,334.0
19532,Rebeka Masarova,Angela Fita Boluda,1.17,4.74,Rebeka Masarova,Rebeka Masarova,Tie,Clay,2025-07-08,Rebeka Masarova,...,4.74,Rebeka Masarova,Angela Fita Boluda,True,False,1.17,4.74,0,2,-100.0
19574,Varvara Gracheva,Margaux Rouvroy,1.21,4.16,Varvara Gracheva,Varvara Gracheva,Tie,Clay,2025-07-09,Varvara Gracheva,...,4.16,Varvara Gracheva,Margaux Rouvroy,True,False,1.21,4.16,0,1,-100.0


In [ ]:
fav_1stwin=data.copy()
fav_1stwin['Set1_fav']=fav_1stwin['set_1_winner']==fav_1stwin['fav']
fav_1stwin=fav_1stwin[fav_1stwin['Set1_fav']==True]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']>1]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']<16]
fav_1stwin=fav_1stwin[fav_1stwin['Surface']=='Clay']
results = fav_1stwin.groupby('fav').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', 'mean')
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.8]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Fav',right_on='fav')
print('Second set selections Favs (if they won 1st set)')
merge[merge['Fav_Odds']>1][['Time','Fav_Odds','Fav','Dog','SecondSet_Percentage']]

In [ ]:
fav_1stloss=data.copy()
fav_1stloss['Set1_fav']=fav_1stloss['set_1_winner']==fav_1stloss['fav']
fav_1stloss=fav_1stloss[fav_1stloss['Set1_fav']==False]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']>1]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']<14]
fav_1stloss=fav_1stloss[fav_1stloss['Surface']=='Clay']
results = fav_1stloss.groupby('fav').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', lambda x: 1 - x.mean())
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.50]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Fav',right_on='fav')
print('Second set selections Favs (if they lost first set)')
merge[merge['Fav_Odds']>1][['Time','Fav_Odds','Fav','Dog','SecondSet_Percentage']]
#fav_1stloss[fav_1stloss['fav']=='Kimmer Coppejans']

In [ ]:
fav_1stwin=data.copy()
fav_1stwin['Set1_dog']=fav_1stwin['set_1_winner']==fav_1stwin['dog']
fav_1stwin=fav_1stwin[fav_1stwin['Set1_dog']==True]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']>1]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']<16]
fav_1stwin=fav_1stwin[fav_1stwin['Surface']=='Clay']
results = fav_1stwin.groupby('dog').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', 'mean')
).reset_index()
results=results[results['Games_Played']>5]
results=results[results['SecondSet_Percentage']>.75]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Dog',right_on='dog')
print('Second set selections Dogs (if they won first set)')
merge[merge['Fav_Odds']>1][['Time','Dog_Odds','Dog','Fav','SecondSet_Percentage']]

In [ ]:
fav_1stloss=data.copy()
fav_1stloss['Set1_dog']=fav_1stloss['set_1_winner']==fav_1stloss['dog']
fav_1stloss=fav_1stloss[fav_1stloss['Set1_dog']==False]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']>1]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']<14]
fav_1stloss=fav_1stloss[fav_1stloss['Surface']=='Clay']
results = fav_1stloss.groupby('dog').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', lambda x: 1 - x.mean())
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.50]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Dog',right_on='dog')
print('Second set selections Dogs (if they lost first set)')
merge[merge['Fav_Odds']>1][['Time','Dog_Odds','Dog','Fav','SecondSet_Percentage']]
#fav_1stloss[fav_1stloss['dog']=='Vit Kopriva']
